# Import Libraries

In [1]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import os

# Define Source

In [2]:
link = 'https://www.nimh.nih.gov/health/topics/autism-spectrum-disorders-asd'

# Read 100 questions list

In [3]:
file = 'Excel/' + os.listdir('Excel')[-1]
print(file)
df = pd.read_excel(file,engine='openpyxl')

df[link]=np.nan
df.head()

Excel/Source8.xlsx


,Question,https://autismnavigator.com/what-is-autism/,https://www.autism.org/is-it-autism/,https://www.autism.org/advice-for-parents/,https://www.autism.org/symptoms-of-autism/,https://www.autism.org/autism-prognosis/,https://autismcanada.org/autism-explained/,https://autismcanada.org/autism-explained/cooccurring-conditions/,https://autismcanada.org/autism-explained/screening-tools/,https://www.who.int/news-room/q-a-detail/autism-spectrum-disorders-(asd),https://www.healthychildren.org/English/health-issues/conditions/Autism/Pages/default.aspx,https://www.ninds.nih.gov/Disorders/Patient-Caregiver-Education/Fact-Sheets/Autism-Spectrum-Disorder-Fact-Sheet#3082_5,https://www.mayoclinic.org/diseases-conditions/autism-spectrum-disorder/symptoms-causes/syc-20352928,https://www.mayoclinic.org/diseases-conditions/autism-spectrum-disorder/diagnosis-treatment/drc-20352934,https://www.necc.org/wp-content/uploads/2015/08/AutismFAQ.pdf,https://www.nimh.nih.gov/health/topics/autism-spectrum-disorders-asd
0,What are the Autism Spectrum Disorders (ASD)?,Autism spectrum disorder (ASD) is a neurodevel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Autism spectrum disorders (ASD) are a group of...,NaN,NaN,OverviewAutism spectrum disorder is a conditio...,NaN,NaN,NaN
1,How is ASD diagnosed?,Diagnosing ASD can be difficult because there ...,NaN,Possible symptoms of autism may be observed by...,NaN,NaN,https://autismcanada.org/autism-explained/diag...,NaN,NaN,NaN,It would be so much easier if \n autism spec...,\nASD symptoms can vary greatly from person to...,NaN,DiagnosisYour child's doctor will look for sig...,NaN,NaN
2,What are early red flags of ASD in toddlers?,The diagnostic features of ASD can be easy to ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,​Many children with \n autism spectrum disor...,NaN,NaN,NaN,NaN,NaN
3,What are 16 early signs of autism by 16 months?,The early signs of autism are easy to miss. Au...,NaN,Autism develops sometime during pregnancy and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,What is Autism?,NaN,Autism Spectrum Disorder (ASD) is a developmen...,Autism is a developmental disability that typi...,NaN,Autism is a neurodevelopmental disorder that b...,"Autism Spectrum Disorder, or autism, is a neur...",NaN,NaN,NaN,NaN,\nAutism spectrum disorder (ASD) refers to a g...,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [4]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(link)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\92.0.4515.107\chromedriver.exe] found in cache


In [5]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [21]:
l1=[]
l2 = []
for i in soup.find_all('section',{'data-cms-anchor':'1'}):
    ques = i.find('h2').text
    ans = i.text
    l1.append(ques)
    l2.append(ans)

In [22]:
l1 = l1[:-2]
l2 = l2[:-2]

In [23]:
len(l1),len(l2)

(5, 5)

In [24]:
for i in range(len(l2)):
    print(i,l1[i])
    print('-'*50)
    print(l2[i])
    print('*'*100)

0 Overview
--------------------------------------------------

Overview

Autism spectrum disorder (ASD) is a developmental disorder that affects communication and behavior. Although autism can be diagnosed at any age, it is said to be a “developmental disorder” because symptoms generally appear in the first two years of life.
According to the Diagnostic and Statistical Manual of Mental Disorders (DSM-5), a guide created by the American Psychiatric Association used to diagnose mental disorders, people with ASD have:
Difficulty with communication and interaction with other people
Restricted interests and repetitive behaviors
Symptoms that hurt the person’s ability to function properly in school, work, and other areas of life
Autism is known as a “spectrum” disorder because there is wide variation in the type and severity of symptoms people experience. ASD occurs in all ethnic, racial, and economic groups. Although ASD can be a lifelong disorder, treatments and services can improve a pers

# Check which questions are similar

In [25]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('autism spectrum disorder','asd',sent) # Compress term
    sent = re.sub('autism','asd',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [26]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [27]:
def sim(li):
    for c1,i in enumerate(li):
        i2 = clean_text(i)
        list1 = i2.split()

        temp1 = 0.0
        temp2 = ''
        temp3 = ''
        temp4 = 0
        temp5 = 0

        for c2,j in enumerate(df['Question']):
            j2 = clean_text(j)
            list2 = j2.split()

            sim = jaccard_similarity(list1, list2)

            if(sim>temp1):
                temp1 = sim
                temp2 = j
                temp3 = i
                temp4 = c2
                temp5 = c1

        if(temp1>0):
            print('Website --> ',temp3,'(Index {})'.format(temp5))
            print('100 questions list --> ',temp2,'(Index {})'.format(temp4))
            print(temp1)
            print('*'*100)

In [28]:
sim(l1)

Website -->  Signs and Symptoms of ASD (Index 1)
100 questions list -->  What are the Signs of Autism? (Index 7)
0.6666666666666666
****************************************************************************************************
Website -->  Causes and Risk Factors (Index 2)
100 questions list -->  Are there risk factors for Autism? (Index 49)
0.5
****************************************************************************************************
Website -->  Diagnosing ASD (Index 3)
100 questions list -->  What is Autism? (Index 4)
0.5
****************************************************************************************************
Website -->  Treatments and Therapies (Index 4)
100 questions list -->  Treatments influence prognosis (Index 31)
0.25
****************************************************************************************************


In [32]:
l1[0]

'Overview'

In [ ]:
# Website, 100Q
# 0, 0
# 1, 7
# 2, 49
# 3, 4
# 4, 46

In [34]:
for i,j in enumerate(df['Question']):
    if('treat' in j.lower()):
        print(i,j)

15 Are there other therapies that are useful in treatment of behavioral symptoms in Autistic children?
31 Treatments influence prognosis
46 
How is autism treated
50 What types of medications are used to treat autistic children?
96 Are medications useful in treating the behavioral problems of Autistic children?
98 Are antipsychotic medications helpful in the treatment of autism?
99 I’ve heard that anti-seizure medication can be helpful in treating some behavior problems associated with autism. Is that true?
100 How are Anxiety and depression treated in an autistic person?
101 What medications are used to treat seizures in autistic children?
102 What are some treatments for involuntary movements, vocalizations and Tourette’s Syndrome associated with Autism?
103 Is there any harm in trying unproven treatments on my child?
105 What are some common dietary interventions used to treat Autism?
106 Is Secretin helpful in treating the symptoms of Autism?
107 Is Naltrexone helpful in treating a

In [36]:
# Directly add answer to existing dataframe
l4 = [[0, 0],
    [1, 7],
    [2, 49],
    [3, 4],
    [4, 46]]

for i in l4:
    df[link].loc[i[1]] = l2[i[0]]

C:\Users\aparg\anaconda3\envs\ASD_scraping\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [37]:
l1

['Overview',
 'Signs and Symptoms of ASD',
 'Causes and Risk Factors',
 'Diagnosing ASD',
 'Treatments and Therapies']

In [38]:
# update questions list
f = [i[0] for i in l4]
f = set(range(len(l1)))-set(f)
l1=[l1[i] for i in f]
l1

[]

In [40]:
df.shape

(146, 16)

# Save Output

In [41]:
df.to_excel('Excel/Source9.xlsx',index=False)